In [79]:
# od pair
from env import env
import pandas as pd
import random as rd

# here requests
from haversine import haversine, Unit
import flexpolyline as fp
import urllib.request
import json, sys

# map plot
from bs4 import BeautifulSoup as bs
import gmplot

In [3]:
df = pd.read_csv('results/bike-results/result.csv')

### Get od pair

In [4]:
# try later: https://geobgu.xyz/py/shapely.html

lat = df['station_lat'].to_list()
lng = df['station_lng'].to_list()

rd.seed(31)
x = rd.sample(range(0,2191),2)

originCoord = (lat[x[0]],lng[x[0]])
origin = f'{originCoord[0]},{originCoord[1]}'

destinationCoord = (lat[x[1]],lng[x[1]])
destination = f'{destinationCoord[0]},{destinationCoord[1]}'

print(origin)
print(destination)

41.9106,-87.6494
41.76,-87.58


### Here requests

Car co2 and fuel consumption link: https://developer.here.com/documentation/routing-api/dev_guide/topics/use-cases/calculate-fuel-consumption.html

In [ ]:
def route_midpoint(route_length,latList,lngList):
    middle = delta = route_length/2 # route length was hereResponse['routes'][0]['sections'][0]['summary']['length']/2
    zipList = list(zip(latList,lngList))
    before = after = None
    dist = 0

    for i in range(len(zipList)-1):
        dist += haversine(zipList[i],zipList[i+1], unit=Unit.METERS)
        sub = abs(middle-dist)
        if sub < delta:
            delta = sub
            before = zipList[i+1]
        if sub > delta:
            after = zipList[i+1]
            break

    midpointCoord = ((before[0]+after[0])/2,(before[1]+after[1])/2)
    midpoint = f'{midpointCoord[0]},{midpointCoord[1]}'

    return midpoint,midpointCoord

# get closest bike share point on coordinate
def closest(coord: list): #### need refactor!!
    data = pd.read_csv('results/bike-results/result.csv')
    dist = sys.float_info.max
    waypoint = ()
    for ind,row in data.iterrows():
        distTest = haversine(coord,(row['station_lat'],row['station_lng']))
        if distTest < dist:
            dist = distTest
            waypoint = (row['station_lat'],row['station_lng'])
    return waypoint

In [42]:
def car_routing(strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    hereKey = env('HERE_API_KEY')
    
    fuelConsumption = '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

    url = 'https://router.hereapi.com/v8/routes?transportMode=%s%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % ('car',fuelConsumption,query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return json.loads(response)


def transit_routing(strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    hereKey = env('HERE_API_KEY')

    # change fuel consumption for transport if possible
    #fuelConsumption = '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

    # multimodal here url
    # transposrt types:
    # vehicle
    # transit
    # taxi
    # rented
    
    url = 'https://intermodal.router.hereapi.com/v8/routes?%s&transit[enable]=entireRoute&rented[enable]=&taxi[enable]=&vehicle[enable]=&alternatives=%d&apiKey=%s' % (query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return json.loads(response)


# for pedestrian: https://developer.here.com/documentation/routing-api/dev_guide/topics/pedestrian-defined.html
def bicycle_routing(strCoord,floatCoord,data,maxAlternatives=1):
    hereKey = env('HERE_API_KEY')

    newOrigin = closest(floatCoord[0]) # data
    newDestination = closest(floatCoord[1]) # data

    mode = ['pedestrian','bicycle','pedestrian']

    query = [
        f'destination={newOrigin[0]},{newOrigin[1]}&origin={strCoord[0]}',
        f'destination={newDestination[0]},{newDestination[1]}&origin={newOrigin[0]},{newOrigin[1]}',
        f'destination={strCoord[1]}&origin={newDestination[0]},{newDestination[1]}'
    ]

    responses = []

    for i in range(len(mode)):
        if mode[i] == 'pedestrian': altNum = 0
        else: altNum = maxAlternatives
        url = 'https://router.hereapi.com/v8/routes?transportMode=%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % (mode[i],query[i],altNum,hereKey)
        print(url)
        response = urllib.request.urlopen(url).read().decode('utf-8')
        responses.append(json.loads(response))

    return responses

In [54]:
#bicycle routing test: ped - bic - ped
response = bicycle_routing((origin,destination),(originCoord,destinationCoord),df)

https://router.hereapi.com/v8/routes?transportMode=pedestrian&destination=41.9106,-87.6494&origin=41.9106,-87.6494&alternatives=0&return=summary,polyline&apikey=RhbiMBCRVNDFplxIdn-yII6eZU9upCqvuChA2n7aSXo
https://router.hereapi.com/v8/routes?transportMode=bicycle&destination=41.76,-87.58&origin=41.9106,-87.6494&alternatives=1&return=summary,polyline&apikey=RhbiMBCRVNDFplxIdn-yII6eZU9upCqvuChA2n7aSXo
https://router.hereapi.com/v8/routes?transportMode=pedestrian&destination=41.76,-87.58&origin=41.76,-87.58&alternatives=0&return=summary,polyline&apikey=RhbiMBCRVNDFplxIdn-yII6eZU9upCqvuChA2n7aSXo


In [59]:
def list_struct(response): # missing add pedestrian to start / end of route
    routes = []

    for i in response:
        aux = []
        for j in i['sections']:
            aux.append((j['transport']['mode'],fp.decode(j['polyline'])))
        routes.append(aux)
    return routes

In [68]:
routes = list_struct(response[1]['routes']) # response[x]['routes']

In [75]:
def map_print(originCoord,routes):

    colors = {
        'pedestrian':'blue',
        'bicycle':'green',
        'car': 'yellow'
    }

    fileName = 0

    outputMap = gmplot.GoogleMapPlotter(originCoord[0],originCoord[1], 12)

    for route in routes:
        for i,j in route:
            lat = [coord[0] for coord in j]
            lng = [coord[1] for coord in j]
            try:
                color = colors[i]
            except:
                color = 'red'
            outputMap.plot(lat, lng, color=color, edge_width=3)

        mapUrl = f'results/map_plot/{fileName}.html'
        fileName += 1

        outputMap.draw(mapUrl)

        # adding the api key into html file
        page = open(mapUrl)
        soup = bs(page.read(),"html.parser")
        srcKey  = f"https://maps.googleapis.com/maps/api/js?key={env('GOOGLE_MAPS_KEY')}&libraries=visualization&sensor=true_or_false"
        line = soup.find('script',type="text/javascript")
        line['src'] = srcKey

        with open(mapUrl, "wb") as f_output:
            f_output.write(soup.prettify("utf-8"))  

In [76]:
map_print(originCoord,routes)

## map test

In [11]:
colors = {
    'pedestrian':'blue',
    'bicycle':'green'
}

def map_print(fileName,originCoord,routes,midpointCoord,pointMarker=False):

    outputMap = gmplot.GoogleMapPlotter(originCoord[0],originCoord[1], 12)

    #for i in range(len(routes)):

    outputMap.plot(latList, lngList, edge_width=3)
    mapUrl = f'results/map_plot/{fileName}.html'

    if pointMarker:
        outputMap.marker(midpointCoord[0],midpointCoord[1],color='green')

    outputMap.draw(mapUrl)

    # adding the api key into html file
    page = open(mapUrl)
    soup = bs(page.read(),"html.parser")
    srcKey  = f"https://maps.googleapis.com/maps/api/js?key={env('GOOGLE_MAPS_KEY')}&libraries=visualization&sensor=true_or_false"
    line = soup.find('script',type="text/javascript")
    line['src'] = srcKey

    with open(mapUrl, "wb") as f_output:
        f_output.write(soup.prettify("utf-8")) 
# map_print('Here_map',originCoord,latList,lngList,midpointCoord)